In [2]:
#!-----extract frequency------
counts = [0] * len(FIELDS)
for d in data:
    for k in d.keys():
        try:
            counts[FIELDS.index(k)]+=1
        except:
            pass

together = []
for i in range(0, len(FIELDS)):
    together += [[counts[i], FIELDS[i]]]

together.sort(reverse=True)

for i in range(0, len(FIELDS)):
    together[i] += [round(together[i][0]/float(len(data))*100,2)]

for i in together:
    print i
#!---------------------------

NameError: name 'FIELDS' is not defined

In [ ]:
for i in range(1,100):
    train_X_2, test_X_2, train_y_2, test_y_2 = cross_validation.train_test_split(X_2, y_2, test_size = i/100.0, random_state = 3)
    regr.fit(train_X_2, train_y_2)
    print i/100.0, regr.score(train_X_2, train_y_2), regr.score(test_X_2, test_y_2)

In [ ]:
#!-----------extract metro ---------
metros = []
for d in data:
    try:
        metros += d['metro']
    except:
        pass

metros = list(set(metros))
metros.sort()
for m in metros:
    print m
#!---------------------------

In [ ]:
#!----------address cleaning------
addresses = []
addr = ['street', 'house_number', 'area', 'district', 'city']
for d in data:
    for field in addr:
        if [d[f] for f in addr].count(d[field]) > 1:
            addresses += [[d[f] for f in addr]]
addresses_strings = []
for a in addresses:
    addresses_strings += [', '.join(a)]

addresses_strings = list(set(addresses_strings))

for a in addresses_strings:
    print a

house_numers = list(set([i['house_number'] for i in data]))

for i in house_numers:
    if re.match(ur"\d+[а-яА-Я\\-]*\d*", i):
        print i
        house_numers.remove(i)

for i in addresses_strings:
    if re.match(ur"\w*\W+, \W+\w*, \W+\w*, Киев", i):
        print i
        addresses_strings.remove(i)

for i in addresses_strings:
    if re.match(ur"\w*\W+, [буд.майд№ ]*\d+[/а-яА-Я\\-]*\d*[а-яА-я/]*, Киев, [буд.майд№ ]*\d+[/а-яА-Я\\-]*\d*[а-яА-я/]*, Киев", i):
        print i
        addresses_strings.remove(i)

for i in addresses_strings:
    if re.match(ur"\w*\W+, [буд.майд№ ]*\d+[/а-яА-Я\\-]*\d*[а-яА-я/]*, \W+\w*, \W+\w*, Киев", i):
        print i
        addresses_strings.remove(i)

for a in addresses_strings:
    a=a.replace(u'a', u'а').replace(u'c', u'с')
#!---------------------------

In [ ]:
#!----------rooms cleaning------
rooms = []
for d in data:
    try:
        if d['rooms'] in [u'91', u'93', u'99']:
            rooms+= [d['code']]
    except:
        pass

big_rooms = []
big_rooms_value = [u'9', u'10', u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'18', u'20', u'21', u'22', u'23', u'24', u'30', u'31', u'33', u'44', u'50', u'56', u'58', u'63', u'68', u'74', u'76', u'81', u'82', u'91', u'93', u'99']

for d in data:
    try:
        if d['rooms'] in big_rooms_value:
            big_rooms+= [d['code']]
    except:
        pass
#!---------------------------

In [ ]:
#!----------square cleaning------
a=get_list_of(data, 'square')
a = [float(i) for i in a]
a.sort()
a = [i for i in a if i >240 or i <30]

square = []
for d in data:
    try:
        if d['square'] in [unicode(i) for i in a]:
            square+= [d['code']]
    except:
        pass

In [ ]:
counter = 0
for d in data:
    try:
        if d['district'].encode('utf-8') in DISTRICT_NAMES_RU:
            counter+= 1
    except:
        pass

In [ ]:
l = len(cleaned_data_2)
for k in range(1, l, 1000):
    cleaned_data_2 = cleaned_data_2[:l-k]
    X_2 = np.array([i[0] for i in cleaned_data_2])
    y_2 = np.array([i[1] for i in cleaned_data_2])
    train_X_2, test_X_2, train_y_2, test_y_2 = cross_validation.train_test_split(X_2, y_2, test_size = 0.18, random_state = 3)
    regr.fit(train_X_2, train_y_2)
    print k, np.mean(abs(regr.predict(test_X_2) - test_y_2)), regr.score(test_X_2, test_y_2)

1.13.1. Removing features with low variance

In [1]:
from sklearn.feature_selection import VarianceThreshold

for p in range(0,100):
    sel = VarianceThreshold(threshold=(p/100.0 * (1 - p/100.0)))
    X_2_new = sel.fit_transform(X_2)    
    print p/100.0, len(X_2_new[0]), process_with(X_2_new, y_2)

KeyboardInterrupt: 

1.13.2. Univariate feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
res = []
for p in range(1,(len(X_2[0])-1)):
    X_2_new = SelectKBest(f_regression, k=p).fit_transform(X_2, y_2)
    res += [process_with(X_2_new, y_2)]
    print len(X_2_new[0]), res[-1] #process_with(X_2_new, y_2)
    
res.index(min(res))

In [ ]:
Feature ranking with recursive feature elimination.

In [ ]:
from sklearn.feature_selection import RFE
# create the RFE model and select 3 attributes
rfe = RFE(regr, 99)
rfe = rfe.fit(X_2, y_2)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
X_2_new=[]
for i in range(0,len(X_2)):
    X_2_new.append([X_2[i][j] for j,el in enumerate(rfe.ranking_) if el==1])
process_with(X_2_new, y_2)

In [ ]:
for k in range(1, len(X_2[0])-1):
    print k
    rfe = RFE(regr, k)
    rfe = rfe.fit(X_2, y_2)
    X_2_new=[]
    for i in range(0,len(X_2)):
        X_2_new.append([X_2[i][j] for j,el in enumerate(rfe.ranking_) if el==1])
    print k, process_with(X_2_new, y_2)

In [ ]:
print ads.count([0.0, 0.0, 0.0]), len(ads)